In [1]:
import os

In [2]:
%pwd

'/home/varun/Documents/TextSummarization/research'

In [3]:
%pwd

'/home/varun/Documents/TextSummarization/research'

In [4]:
os.chdir("/home/varun/Documents/TextSummarization")

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.textSummarizer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.textSummarizer.utils.common import read_yaml, create_directories

In [7]:
CONFIG_FILE_PATH

PosixPath('config/config.yaml')

In [8]:
PARAMS_FILE_PATH

PosixPath('params.yaml')

In [9]:
read_yaml(CONFIG_FILE_PATH)

[2024-03-18 14:41:28,359: INFO: common: yaml file: config/config.yaml loaded successfully]


ConfigBox({'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip', 'local_data_file': 'artifacts/data_ingestion/summarizer-data.zip', 'unzip_dir': 'artifactsdata_ingestion'}})

In [18]:
class ConfigurationManager:
    def __init__(self, 
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config
        

In [19]:
import os
import urllib.request as request
import zipfile
from src.textSummarizer.logging import logger
from src.textSummarizer.utils.common import get_size

In [20]:
class DataIngesion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
            
    def extract_zip_file(self):
        
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        

        

In [21]:
try:
    config = ConfigurationManager()
    data_ingestion_config  = config.get_data_ingestion_config()
    data_ingestion = DataIngesion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-03-18 14:52:06,156: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-18 14:52:06,158: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-18 14:52:06,158: INFO: common: created directory at: artifacts]
[2024-03-18 14:52:06,159: INFO: common: created directory at: artifacts/data_ingestion]
[2024-03-18 14:52:06,160: INFO: 2459005936: File already exists of size: ~7718 KB]
